# ANOVA

In [2]:
import pandas as pd
import pingouin as pg
import seaborn as sns
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

A glass of beer with the head having receded, leaving behind noticeable lacing

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Mai_2012_Weißbier.JPG/1920px-Mai_2012_Weißbier.JPG" width="200"/>

Image credit: Mai Weißbier, 2012

## Beer Head Retention

There's nothing like a good foamy head on a cold beer, but [it needs to be just right](https://en.wikipedia.org/wiki/Beer_head): not too much, not too little. That means there is a financial incentive for brewers to get the foam just right every time, and that means: science! Below, you can read the abstract from a study by Ault et al (1967), who devised a means to measure beer head retention (how much foam there is, how long it lasts, and how much it sticks to the glass) in a way that would as closely as possible mimic the consumer's experience.

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Beer%20Head%20Retention/image_abstract.png?raw=true" width="500"/>

Below, you can see a schematic drawing of the device used for measuring beer head retention.

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Beer%20Head%20Retention/image_head_retention_apparatus.png?raw=true" width="400"/>

Ault et al (1967) measured [sixty bottles of beer](https://en.wikipedia.org/wiki/99_Bottles_of_Beer) from three different bottlings: 

1. Bottling: the batch of beers bottled at the same time
2. Bottle: the bottle id number for each bottling
3. Head_Formation: the distance from the top of the glass to the top of the foam after 5 seconds plus the distance from the previous top of the foam to the top of the foam after 30 seconds (see Ault et al, 1967, for details)
4. Percent_Adhesion: The amount of "lacing", that is, foam stuck to the glass after the foam collapses
5. Percent_Collapse: Essentially, how much foam is left after 240 seconds.

Assignment: 

1. Compare the different bottling batches on each of the three outcome measures: Head_Formation, Percent_Adhesion, and Percent_Collapse, and report your results in APA format. Using the names from ``pingouin`` output, this could be "A one-way ANOVA showed a significant effect of / no significant effect of SOURCE on DV (F(DDOF1, DDOF2) = F, p = p".
2. Make figures illustrating your results
3. If you drink beer, based on your own preferences, which bottling had the best balance of formation, adhesion, and collapse?

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/ethanweed/datasets-for-teaching/main/Beer%20Head%20Retention/beer-head-data.csv")

## Barking sea lions

California sea lion (Zalophus californianus) and a Western gull (Larus occidentalis) in Scripps Park in La Jolla (San Diego, California).

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/California_sea_lion_in_La_Jolla_%2870568%29.jpg/2560px-California_sea_lion_in_La_Jolla_%2870568%29.jpg" width="300"/>

Image credit: Rhododendrites (2016)

Sea lions love to make noise, and in the sea lion business, the noise they make is called "barking". To the untrained ear, all sea lions may sound the same, but in fact, there may be local "dialects". Ahonen et al (2014) carried out a study of sea lion barks, measuring, among other things, the duration of the barks at seven different locations in Australia (see map below).

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Durations%20of%20Seal%20Lion%20Barking%20Calls/image_locations.png?raw=true" width="550"/>

Below, you can read the article abstract:

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Durations%20of%20Seal%20Lion%20Barking%20Calls/image_abstract.png?raw=true" width=""/>

Assignment:

1. Use the dataset below to check the researchers' results. Was there a signficant difference in barking duration between the different locations?
2. Report your findings using the APA format (see beer example above)
3. Reproduce panel A from figure 4, below. Sadly, we don't have the data for the interbark durations, so you can't do panel B. You can use the location numbers, but for an extra challenge, use Python to replace the numbers with the location names. You can see the location names in table 2 below, which are ordered in the same order as the location numbers in the data:

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Durations%20of%20Seal%20Lion%20Barking%20Calls/image_table2.png?raw=true" width=""/>

<img src="https://github.com/ethanweed/datasets-for-teaching/blob/main/Durations%20of%20Seal%20Lion%20Barking%20Calls/image_figure4.png?raw=true" width=""/>

## References

Ahonen, H., Stow, A. J., Harcourt, R. G., & Charrier, I. (2014). Adult male Australian sea lion barking calls reveal clear geographical variations. Animal Behaviour, 97, 229-239.

Ault, R. G., Hudson, E. J., Linehan, D. J., & Woodward, J. D. (1967). A practical approach to the assessment of head retention of bottled beers. Journal of the Institute of Brewing, 73(6), 558-566.



## Lexical Priming

In 2021, Peel et al studied the effects of word repetition, case, and stimulus onset asynchrony in a semantic categorization task. The abstract for the paper is below:

<img src="https://github.com/ethanweed/ExPsyLing/blob/master/Experiments/Experiment%201/Peel_abstract.png?raw=true" width=""/>



In 2024, a group of researchers in Aarhus, Denmark conducted a partial replication study, in which they used the same materials to test the effects of word repetition and soa on response time and accuracy in a semantic categorization task. As in Peel et (2021), participants were asked to categorize a word flashed on the screen as either natural or man-made. Each word was preceeded by a prime, which was either the same word (congruent condition) or a different word (incongruent condition). Stimuli were furthermore presented in two blocks, with either short (immediate presentation) or long (80 ms delay) stimulus onset asynchrony (SOA). All participants completed each block.

Use a repeated-measures ANOVA to test whether there was:

1. A main effect of repetition
2. A main effect of SOA
3. An interaction between repetition and SOA

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/ethanweed/ExPsyLing/refs/heads/master/Experiments/Experiment%201/data/data_ExPsyLing_2024.csv')
df.head()

,url,sender,sender_type,sender_id,ended_on,duration,time_run,time_render,time_show,time_end,...,response,response_action,prime,target,condition,soa_condition,congruence,correct_response,correctResponse,correct
0,4482,Wecome0,canvas.Screen,0,timeout,500.0,1198.0,1199.0,1217.0,1701.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4482,Wecome1,canvas.Screen,1,timeout,500.0,1705.0,1701.0,1717.0,2201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4482,Wecome2,canvas.Screen,2,timeout,500.0,2202.0,2201.0,2217.0,2701.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4482,Wecome3,canvas.Screen,3,timeout,500.0,2702.0,2701.0,2717.0,3201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4482,Instructions1,canvas.Screen,4,response,13495.0,3202.0,3201.0,3217.0,16696.0,...,NaN,keypress,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Peel et al (2021) present their main results in a figure, seen below:

<img src="https://github.com/ethanweed/ExPsyLing/blob/master/Experiments/Experiment%201/Peel-et-al_2022-results.png?raw=true" width=""/>

The Aarhus researchers did not collect data measuring the effect of same vs. different case (the primes were always in a lower case, and the target words in upper case), so we cannot reproduce the entire figure. To the best of your ability, however, try to reproduce Peel et al's figure 1, panels a and c.